# Assess portfolio
ML for trading Udacity Course exercise

Analyze a portfolio.
http://quantsoftware.gatech.edu/Optimize_something

A transcription of the Udacity Course lectures can be find on https://docs.google.com/document/d/1ELqlnuTSdc9-MDHOkV0uvSY4RmI1eslyQlU9DgOY_jc/edit?usp=sharing

Kairoart 2018
"""


## Overview

In this project you will use what you learned about optimizers to optimize a portfolio. That means that you will find how much of a portfolio's funds should be allocated to each stock so as to optimize it's performance. We can optimize for many different metrics. In previous versions of this assignment, we tried to maximize Sharpe Ratio. We're changing it this time to define "optimal" as minimum volatility.

You will leverage the functions you created in the "asess_portfolio" project that assessed the value of a portfolio with a given set of allocations.

## Task
Implement a Python function named optimize_portfolio() that can find the optimal allocations for a given set of stocks. You should optimize for minimum volatility (i.e., standard deviation of daily return).

The function should accept as input a list of symbols as well as start and end dates and return a list of floats (as a one-dimensional numpy array) that represents the allocations to each of the equities. You should take advantage of routines developed in the portfolio analysis project to compute daily portfolio value and statistics. You should cut-and-paste your code for the functions that did this from the last project into optimization.py.

You are given the following inputs for optimizing a portfolio:

* A date range to select the historical data to use (specified by a start and end date)
* Symbols for equities (e.g., GOOG, AAPL, GLD, XOM). Note: You should support any symbol in the data directory.

You should implement the following API EXACTLY.

import datetime as dt
allocs, cr, adr, sddr, sr = \
    optimize_portfolio(sd=dt.datetime(2008,1,1), ed=dt.datetime(2009,1,1), \
    syms=['GOOG','AAPL','GLD','XOM'], gen_plot=False)


### Inputs

* sd: A datetime object that represents the start date
* ed: A datetime object that represents the end date
* syms: A list of symbols that make up the portfolio (note that your code should support any symbol in the data directory)
* gen_plot: If True, create a plot named plot.png

### Output

* allocs: A 1-d Numpy ndarray of allocations to the stocks. All the allocations must be between 0.0 and 1.0 and they must sum to 1.0.
* cr: Cumulative return
* adr: Average daily return
* sddr: Standard deviation of daily return
* sr: Sharpe ratio

## Goal

Your goal is to find allocations to the symbols that optimize the criteria given above. Assume 252 trading days in a year and a risk free return of 0.0 per day.

## Import libraries

In [29]:
import numpy as np
import pandas as pd
import scipy
import os
import pandas.io.data as web
import matplotlib.pyplot as plt
import scipy.optimize as sco

# Add parent directory PATH for looking for modules,
import sys
sys.path.insert(0,'..')
from util import get_data, plot_data

## Define functions

### Find optimal allocations

* We have to provide a function to minimize f(x) that takes in x. min_func_sharpe()
* An initial guess for that x. 
* Call the optimizer and let it run.


In [ ]:
def find_optimal_allocations(prices):

    noa = len(prices.columns)
    rets = np.log(prices / prices.shift(1))
    #print rets

    # Expected portfolio returns
    prets = [0, 0, 0, 0]
    # Expected portfolio volatility    
    pvols = [0, 0, 0, 0]
    
    for p in range (2500):
        allocs = np.random.random(noa)
        allocs /= np.sum(allocs)
        prets.append(np.sum(rets.mean() * allocs) * 252)
        pvols.append(np.sqrt(np.dot(allocs.T, np.dot(rets.cov() * 252, allocs))))
    
    
    prets = np.array(prets)
    pvols = np.array(pvols)

    def statistics(allocs):
        ''' Returns portfolio statistics.
        Parameters
        ==========
        weights : array-like
        weights for different securities in portfolio
        Returns
        =======
        pret : float
        expected portfolio return
        pvol : float
        expected portfolio volatility
        pret / pvol : float
        Sharpe ratio for rf=0
        '''
        allocs = np.array(allocs)
        pret = np.sum(rets.mean() * allocs) * 252
        pvol = np.sqrt(np.dot(allocs.T, np.dot(rets.cov() * 252, allocs)))
        return np.array([pret, pvol, pret / pvol])


    def min_func_sharpe(allocs):
        return -statistics(allocs)[2]

    # Constrains
    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    # Ranges
    bnds = tuple((0, 1) for x in range(noa))

    opts = sco.minimize(min_func_sharpe, noa * [1. / noa,], method='SLSQP', bounds=bnds, constraints=cons)

    allocs = opts['x']
    return allocs



In [195]:
# This is the function that will be tested by the autograder
# The student must update this code to properly implement the functionality
def optimize_portfolio(sd = dt.datetime(2008,1,1), ed = dt.datetime(2009,1,1), \
    syms = ['GOOG','AAPL','GLD','XOM'], gen_plot=False):

    # Read in adjusted closing prices for given symbols, date range
    dates = pd.date_range(sd, ed)
    prices_all = get_data(syms, dates)  # automatically adds SPY
    prices = prices_all[syms]  # only portfolio symbols
    prices_SPY = prices_all['SPY']  # only SPY, for comparison later

    
    # Get daily portfolio value
    # 1. Normalize the prices Dataframe
    normed = pd.DataFrame(index=prices.index)
    for column in prices:
        normed[column] = prices[column].values / prices[column].iloc[0];
    #print(normed)

    # 2. Calculate allocs
    noa = len(symbols)
    alloced = np.random.random(noa)
    alloced /= np.sum(alloced)
    alloced = find_optimal_allocations(prices) #optimize allocations
    alloced = alloced / np.sum(alloced)  # normalize allocations, if they don't sum to 1.0

    # 3. pos_vals = alloced * start vals Dataframe
    pos_vals = pd.DataFrame()
    for column in alloced:
        pos_vals[column] = alloced[column] * sv
    pos_vals['pos_vals'] = pos_vals.iloc[:,-4:].sum(axis=1)
    #print(pos_vals)


    # 4. port_val Series
    port_val = pd.Series(pos_vals.pos_vals)
    #print(port_val)

    # 5. Daily returns Series
    daily_rets = port_val.pct_change(1)
    #print(daily_rets)
    #print (type(daily_rets))

    # Get portfolio statistics (note: std_daily_ret = volatility)
    # 1. Cumulative return
    cr = (port_val[-1] / port_val[0] -1)
    
    # 2. Averrage daily returns
    adr = daily_rets.mean()
    # 3. Standard deviation of daily returns
    sddr = daily_rets.std()
    # 4. Sharpe ratio
    sr = adr / sddr
    k = math.sqrt(252)
    sr = k * sr
    
    # Compare daily portfolio value with SPY using a normalized plot
    if gen_plot:
        # Normalize port_val
        x = port_val[0]
        port_val = port_val.divide(x, fill_value=1)

        # Normalize SPY
        x = prices_SPY[0]
        prices_SPY = prices_SPY.divide(x, fill_value=1)

        df_temp = pd.concat([port_val, prices_SPY], 
                             keys=['Portfolio', 'SPY'], axis=1)
        df_temp.plot()
        
    #print(df_temp)
    

    return cr, adr, sddr, sr

def test_code():
    # This code WILL NOT be tested by the auto grader
    # It is only here to help you set up and test your code

    # Define input parameters
    # Note that ALL of these values will be set to different values by
    # the autograder!
    start_date = dt.datetime(2010,1,1)
    end_date = dt.datetime(2010,12,31)
    symbols = ['GOOG', 'AAPL', 'GLD', 'XOM']
    allocations = [0.2, 0.3, 0.4, 0.1]
    start_val = 1000000  
    risk_free_rate = 0.0
    sample_freq = 252

    # Assess the portfolio
    cr, adr, sddr, sr, ev = assess_portfolio(sd = start_date, ed = end_date,\
        syms = symbols, \
        allocs = allocations,\
        sv = start_val, \
        gen_plot = True)

    # Print statistics
    print("Start Date:", sd)
    print("End Date:", ed)
    print("Symbols:", syms)
    print("Allocations:", allocs)
    print("Sharpe Ratio:", sr)
    print("Volatility (stdev of daily returns):", sddr)
    print("Average Daily Return:", adr)
    print("Cumulative Return:", cr)